<a href="https://colab.research.google.com/github/ii200400/Tensorflow_Tutorial/blob/master/05%20-%20TensorBoard%2C%20Saver/01_Saver.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

이번 실습은 구글 드라이브를 연동하여 파일을 저장하고 불러올 필요가 있기 때문에 아래의 코드를 추가하고 data.scv의 파일 경로를 바꿔주었다.

In [ ]:
# 구글 드라이브 연동을 위한 import
from google.colab import drive

# 구글 드라이브 연동을 위한 인증
drive.mount('/content/drive')

basic_path = 'drive/My Drive/Colab Notebooks/3분 딥러닝 텐서플로맛/05 - TensorBoard, Saver'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# 개요

학습이 된 모델을 저장하고 재사용을 위해서 불러오기

In [ ]:
%tensorflow_version 1.x
import tensorflow as tf
import numpy as np

## 데이터 정의

다른 때와 다르게 데이터를 scv파일에서 불러온다.\
csv는 입력값 저장하는 파일형식으로 자주쓰이는 것 같다.

또한 여기에서는 numpy 라이브러리와 여러 매개변수로 데이터를 가져왔지만\
나중에는 더 편리한 라이브러리로 매개변수 없이도 쉽게 데이터를 조작할 것이다.

In [ ]:
# np.loadtxt의 매개변수 unpack과 np.transpose는
# 주어진 행렬을 전치행렬로 만들어 반환한다.

data = np.loadtxt(basic_path + '/data.csv', delimiter=',', unpack=True, dtype='float32')

# 털, 날개, 기타, 포유류, 조류
# x_data = 0, 1
# y_data = 2, 3, 4
x_data = np.transpose(data[0:2])
y_data = np.transpose(data[2:])

## 모델 구성

### 변수와 가중치 정의

In [ ]:
# 학습에 직접적으로 사용하지 않고 학습 횟수에 따라 단순히 증가시킬 변수를 만든다.
# 때문에 trainable 매개변수를 False로 지정한다.
global_step = tf.Variable(0, trainable=False, name='global_step')

X = tf.placeholder(tf.float32)
Y = tf.placeholder(tf.float32)

W1 = tf.Variable(tf.random_uniform([2, 10], -1., 1.))
W2 = tf.Variable(tf.random_uniform([10, 20], -1., 1.))
W3 = tf.Variable(tf.random_uniform([20, 3], -1., 1.))

### 신경망 구성 및 비용 정의

In [ ]:
L1 = tf.nn.relu(tf.matmul(X, W1))
L2 = tf.nn.relu(tf.matmul(L1, W2))

model = tf.matmul(L2, W3)

cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels=Y, logits=model))

### 최적화 함수

In [ ]:
# 최적화 함수는 아담을 사용한다.
optimizer = tf.train.AdamOptimizer(learning_rate=0.01)
# 학습용 변수들을 최적화 할 때 마다 global_step로 넘겨준 변수가 1씩 증가된다.
train_op = optimizer.minimize(cost, global_step=global_step)

## 모델 학습 및 테스트

### 모델 불러오기

In [ ]:
sess = tf.Session()
# 모델을 저장하고 불러오는 API를 초기화한다.
# global_variables 함수를 통해 앞서 정의하였던 변수들을 가져온다.
# 해당 함수로 사용한 변수들을 저장하고 불러올 수 있도록 만들 수 있다.
saver = tf.train.Saver(tf.global_variables())

# model파일에 저장된 체크포인트가 있다면
ckpt = tf.train.get_checkpoint_state(basic_path + '/model')
if ckpt and tf.train.checkpoint_exists(ckpt.model_checkpoint_path):
  # 그 체크포인트를 불러오고
  saver.restore(sess, ckpt.model_checkpoint_path)
else: # 없다면
  # 변수를 초기화하는 함수를 호출한다.
  sess.run(tf.global_variables_initializer())

Instructions for updating:
Use standard file APIs to check for files with this prefix.
INFO:tensorflow:Restoring parameters from drive/My Drive/Colab Notebooks/3분 딥러닝 텐서플로맛/05 - TensorBoard, Saver/model/dnn.ckpt-2


### 모델 학습 및 저장

In [ ]:
for step in range(2):
    sess.run(train_op, feed_dict={X: x_data, Y: y_data})

    print('Step: %d, ' % sess.run(global_step),
          'Cost: %.3f' % sess.run(cost, feed_dict={X: x_data, Y: y_data}))

# 학습 후 변수를 저장한다.
saver.save(sess, basic_path + '/model/dnn.ckpt', global_step=global_step)

Step: 3,  Cost: 1.139
Step: 4,  Cost: 1.108


'drive/My Drive/Colab Notebooks/3분 딥러닝 텐서플로맛/05 - TensorBoard, Saver/model/dnn.ckpt-4'

### 모델 테스트

In [ ]:
# 결과 확인
# 0: 기타 1: 포유류, 2: 조류

prediction = tf.argmax(model, 1)
target = tf.argmax(Y, 1)
print('예측값:', sess.run(prediction, feed_dict={X: x_data}))
print('실제값:', sess.run(target, feed_dict={Y: y_data}))

is_correct = tf.equal(prediction, target)
accuracy = tf.reduce_mean(tf.cast(is_correct, tf.float32))
print('정확도: %.2f' % sess.run(accuracy * 100, feed_dict={X: x_data, Y: y_data}))

예측값: [0 2 2 0 0 1]
실제값: [0 1 2 0 0 2]
정확도: 66.67


# 전체 코드

In [ ]:
%tensorflow_version 1.x
import tensorflow as tf
import numpy as np

data = np.loadtxt(basic_path + '/data.csv', delimiter=',',
                  unpack=True, dtype='float32')

# 털, 날개, 기타, 포유류, 조류
x_data = np.transpose(data[0:2])
y_data = np.transpose(data[2:])


#########
# 신경망 모델 구성
######
global_step = tf.Variable(0, trainable=False, name='global_step')

X = tf.placeholder(tf.float32)
Y = tf.placeholder(tf.float32)

W1 = tf.Variable(tf.random_uniform([2, 10], -1., 1.))
L1 = tf.nn.relu(tf.matmul(X, W1))

W2 = tf.Variable(tf.random_uniform([10, 20], -1., 1.))
L2 = tf.nn.relu(tf.matmul(L1, W2))

W3 = tf.Variable(tf.random_uniform([20, 3], -1., 1.))
model = tf.matmul(L2, W3)

cost = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits_v2(labels=Y, logits=model))

optimizer = tf.train.AdamOptimizer(learning_rate=0.01)
train_op = optimizer.minimize(cost, global_step=global_step)


#########
# 신경망 모델 학습
######
sess = tf.Session()
saver = tf.train.Saver(tf.global_variables())

ckpt = tf.train.get_checkpoint_state(basic_path + '/model')
if ckpt and tf.train.checkpoint_exists(ckpt.model_checkpoint_path):
  saver.restore(sess, ckpt.model_checkpoint_path)
else:
  sess.run(tf.global_variables_initializer())

for step in range(2):
  sess.run(train_op, feed_dict={X: x_data, Y: y_data})

  print('Step: %d, ' % sess.run(global_step),
        'Cost: %.3f' % sess.run(cost, feed_dict={X: x_data, Y: y_data}))

saver.save(sess, basic_path + '/model/dnn.ckpt', global_step=global_step)

#########
# 결과 확인
# 0: 기타 1: 포유류, 2: 조류
######
prediction = tf.argmax(model, 1)
target = tf.argmax(Y, 1)
print('예측값:', sess.run(prediction, feed_dict={X: x_data}))
print('실제값:', sess.run(target, feed_dict={Y: y_data}))

is_correct = tf.equal(prediction, target)
accuracy = tf.reduce_mean(tf.cast(is_correct, tf.float32))
print('정확도: %.2f' % sess.run(accuracy * 100, feed_dict={X: x_data, Y: y_data}))

Step: 1,  Cost: 1.206
Step: 2,  Cost: 1.172
예측값: [0 2 2 0 0 1]
실제값: [0 1 2 0 0 2]
정확도: 66.67
